## Ogólne sformułowanie problemu
Oznaczmy przez Y obserwowaną zmienną losową (potencjalnie wielowymiarową) o rozkładzie F z rodziny indeksowanej parametrem $\theta$. Bezpośrednia estymacja $\theta$ metodą największej wiarygodności jest trudna, wiec rozszerzamy przestrzeń o zmienne ukryte/nieobserwowane $Z$.

Łączny wektor zmiennych losowych $(Y, Z)$ pochodzi z rodziny indeksowanej parametrem $\theta \in \Theta$. Gęstość tego rozkładu oznaczamy przez $f(y, z;\theta)$.

## Algorytm EM
Algorytm EM ma następującą postać:
1. Wybierz początkową wartość $\hat{\theta}^{(1)}$,
2. Krok E (Expectation). Wyznaczamy warunkową wartość oczekiwaną dla aktualnego $\hat{\theta}^{(j)}$, gdzie $j$ oznacza numer iteracji.
$$Q(\theta, \hat{\theta}^{(j)})=E(l(\theta; Z, Y)|Y,  \hat{\theta}^{(j)})$$
Będziemy ją traktować jako funkcję po $\theta$.
3. Krok M (Maximization). Wyznacz kolejną wartość $\hat{\theta}^{(j+1)}$, taką która maksymalizuje $Q(\theta, \hat{\theta}^{(j)})$ po $\theta$
$$\hat{\theta}^{(j+1)}$=arg \max\limits_{\theta} Q(\theta, \hat{\theta}^{(j)}) $$
4. Powtarzaj kroki 2-3 do spełnienia określonego warunku stop.


### Dlaczego to działa?
Pytanie na które musimy odpowiedzieć, to dlaczego optymalizacja $Q(\theta, \hat{\theta}^{(j)})$ pomaga w maksymalizacji funkcji wiarygodności.

Zacznijmy od rozkładu warunkowego
$$P(Z|Y;\theta)=\frac{P(Z,Y;\theta)}{P(Y;\theta)}$$
a więc
$$P(Y;\theta)=\frac{P(Z,Y;\theta)}{P(Z|Y;\theta)}$$

Interesuje nas funkcja log-wiarygodnosci. Korzystając z powyższego, można ją wyrazić jako:

$$l(\theta;Y)=l_0(\theta; Z,Y)-l_1(\theta;Z|Y)$$

Zauważmy, że funkcje wiarygodności $l()$, $l_0()$ i $l_1()$  dotyczącą różnych zmiennych, stąd różne oznaczenia.

Nałóżmy obustronnie warunkową wartość oczekiwaną $E(.|Y, \theta')$. Parametr $\theta'$ będziemy w przyszłości wykorzystywali jako ustalony, roboczy, suboptymalny wektor parametrów. Otrzymujemy

$$l(\theta; Y) = E(l_0(\theta; Z, Y)|Y, \theta') - E(l_1(\theta; Z|Y)|Y, \theta')$$

Aby uprościć zapis poniżej, przyjmijmy takie standardowe oznaczenie dla prawej strony tego równania.

$$l(\theta; Y) = Q(\theta, \theta') - R(\theta, \theta')$$

Rozłożyliśmy funkcję log-wiarygodności, którą chcieliśmy maksymalizować po $\theta$ na dwa człony. Algorytm EM będzie bezpośrednio maksymalizował $Q(\theta, \theta')$.

Funkcja $R(\theta, \theta')$ ma swoje maksimum, gdy wartość oczekiwana jest warunkowana tym samym rozkładem co będąca pod nią funkcja wiarygodności. Jest tak dlatego, że funkcjąa wiarygodności jest wypukła i można wykorzystać nieróœność Jensena.

A więc $R(\theta, \theta') \leq R(\theta', \theta')$, co z kolei oznacza, że jeżeli jesteśmy w stanie znaleźć $\theta$, które zwiększy $Q(\theta, \theta')$ to z pewnością zwiększymy tez $l(\theta; Y)$.

## Przykład

Ogólne sformułowanie algorytmu EM umożliwia stosowanie go do rozmaitych sytuacji. Ale aby wyrobić sobie intuicję przeliczmy jeden prosty przykład, jednowymiarowej mieszaniny dwóch rozkładów normalnych różniących się tylko średnią. Dla większej liczby składowych, wymiarów i parametrów obliczenia przeprowadza się według tego samego schematu.

Model mieszaniny. Dwie składowe:

$$Y_1~\mathcal{N}(\mu_1, \sigma^2)$$
$$Y_2~\mathcal{N}(\mu_2, \sigma^2)$$

rozkład mieszający

$$Z ~ B(1,\pi)$$

i mieszanina

$$Y=(1-Z)Y_1+ZY_2$$

Mieszanina to w $\pi$ rozkład $Y_1$ a w $1-\pi$ rozkład zmiennej $Y_2$. Gęstość tego rozkładu można opisać jako

$$f_Y(x) = (1-\pi) \phi_1(x)+\pi\phi_2(x)$$

Nasz wektor parametrów to $\theta = (\pi, \mu_1, \mu_2, \sigma) a funkcja log-wiarygodność ma postać

$$l(\theta, y) = \sum_i log[(1-\pi)\phi_1(y_i)+\pi\phi_2(y_2)]$$

Optymalizacja takiej sumy logarytmów nie jest prosta.

Zgodnie z duchem EM ułatwimy sobie zadanie dodając do modelu zmienne ukryte Z. Funkcja wiarygodnosci w takim modelu ma postać

$$l(\theta, y, z) = \sum_i[(1-z_i)log\phi_1(y_i)+z_ilog\phi_2(y_i)]+\sum_i[(1-z_i)log\pi+z_i log(1-\pi)]$$

Tę funkcję bedzie nam łatwiej maksymalizować. Przyjrzyjmy się krokom w algorytmie EM.


### Krok E

Chcemy wyznaczyć warunkową wartość oczekiwanej funkcji $l(\theta, y, z)$ po $y, \theta^{(i)}$. Człony funkcji wiarygodności zawierające $y_i$ się nei zmieniają. Musimy jedynie policzyć co się stanie z członem $z_i$.

$$E(Z_i|Y;\theta^{(i)})=\frac{\hat{\pi}^{(i)}\phi_2(y_i)}{(1-\hat{\pi}^{(i)})\phi_1(y_i)+\hat{\pi}^{(i)}\phi_2(y_i)}=\eta_i$$

Wyliczone wartości oczekiwane wstawiamy w miejsce $z_i$.

### Krok M

Funkcja $Q(\theta, \hat{\theta}^{(i))}$ jest już funkcją parametrów $(\mu_1, \mu_2, \sigma, \pi)$. Z uwagi na jej postać, można każdy z parametrów maksymalizować niezależnie. Wyznaczamy pochodną i przyrównujemy do zera.

Otrzymamy

$$\hat{\mu}_1^{(j+1)}=\frac{\sum_i(1-\eta_i)y_i}{\sum_i(1-\eta_i)}$$

$$\hat{\mu}_2^{(j+1)}=\frac{\sum_i\eta_iy_i}{\sum_i\eta_i}$$

$$\hat{\sigma}^{2,(j+1)}_2 = \Big[\sum_i(1-\eta_i)(y_i-\hat{\mu}_1^{(j)})^2+\sum_i\eta_i(y_i-\hat{\mu}_2^{(j)})^2\Big]/n$$

$$\hat{\pi}^{(j+1)}=\sum_i\eta_i/n$$


Kroki E i M należy powtarzać aż nie uzyska się przyzwoitej zbieżności.


## Estymacja MAP

Przedstawiony powyżej algorytm EM maksymalizuje funkcje wiarygodności. Rosnąca popularność metoda Bayesowskich spowodowała, że jego odmiana jest też wykorzystywana do znajdowania estymatorów maksimum a-posteriori (MAP).

Punkstem wyjścia jest wzór na rozkład a-posteriori
$$p(\theta|Y)\approx l(\theta, y)+log(p(\theta))$$

Algorytm EM jest ogólnym algorytmem maksymalizacji. Możemy go wykorzystywać również do szukania maksimum rozkładu aposteriori, dodając w kroku E człon $log(p(\theta))$.

$$Q^{MAP}(\theta,\theta')=Q(\theta,\theta')+log(p(\theta))$$

## Inne linki
https://people.duke.edu/~ccc14/sta-663/EMAlgorithm.html
http://www.nature.com/nbt/journal/v26/n8/fig_tab/nbt1406_F1.html